In [1]:
rm(list=ls())
# William: commented out below for notebook
# knitr::opts_chunk$set(echo = TRUE)
getwd()
options(scipen = 10)
# William: prevent excessive verbosity
suppressMessages(
  library(tmle)
  )
# William: prevent excessive verbosity
suppressMessages(
  library(ggplot2)
)
library(SuperLearner)
# William: prevent excessive verbosity
suppressMessages(
  library(dplyr)
  )
library(magrittr)
library(knitr)
library(foreign)
library(ck37r)
suppressMessages(
  library(sl3)
  )
suppressMessages(
  library(arm)
  )
# Prevent verbosity from caret
library(lattice)
library(caret)
suppressMessages(
  library(data.table)
  )
library(screening)
# William: added this line so that SuperLearner calls will work
suppressMessages(
  library(xgboost)
  )
# William: added these two lines to prevent unnecessary warnings when screening() is called
library(foreach)
library(glmnet)


[1] "/Users/williamkrinsman/Repositories/Causal_Uganda"

Loaded glmnet 2.0-13



In [2]:
data <- read.dta("Data/SEED_endline_analysis.dta", convert.factors = FALSE, convert.underscore = FALSE)
data <- data.frame(data)

In [3]:
# List to hold the different column names.
(names = list(
  # Outcomes of interest
  outcome = c("ever_self_employed","log_tot"),

  # Treatment variable
  treatment = "treated",

  # Adjustment covariates
  covars = c("treated","gender","age","q06_dayorboarding","q25_family_business","q25a_wk_family_bus","timeprefs_patience","riskbehavior","mathbusiness","leadership","perceivedcontrol","timeprefs_delta","timeprefs_beta","prosocialbehavior","anxiety","selfconfidence","big5extroversion","big5emostability","big5openness","big5conscientious","big5agreeable","schoolacceptance","currfamwealthstep","tenyrwealthstep","takingriskstep","ravenscore","father_educ2","father_educ3","father_educ4","father_educ5","father_income2","father_income3","mother_income2","mother_income3","type_house","q13_olevelscore2","q13_olevelscore34")
))

$outcome
[1] "ever_self_employed" "log_tot"           

$treatment
[1] "treated"

$covars
 [1] "treated"             "gender"              "age"                
 [4] "q06_dayorboarding"   "q25_family_business" "q25a_wk_family_bus" 
 [7] "timeprefs_patience"  "riskbehavior"        "mathbusiness"       
[10] "leadership"          "perceivedcontrol"    "timeprefs_delta"    
[13] "timeprefs_beta"      "prosocialbehavior"   "anxiety"            
[16] "selfconfidence"      "big5extroversion"    "big5emostability"   
[19] "big5openness"        "big5conscientious"   "big5agreeable"      
[22] "schoolacceptance"    "currfamwealthstep"   "tenyrwealthstep"    
[25] "takingriskstep"      "ravenscore"          "father_educ2"       
[28] "father_educ3"        "father_educ4"        "father_educ5"       
[31] "father_income2"      "father_income3"      "mother_income2"     
[34] "mother_income3"      "type_house"          "q13_olevelscore2"   
[37] "q13_olevelscore34"

In [4]:
# Keep variables of interest

data <- subset(data, select = c(names$outcome, names$treatment, names$covars))
# Review missing values in id, outcome, treatment, and censoring variables.
# Outcome is the only variable that can have missing values.
colSums(is.na(data[, c(names$outcome, names$censoring, names$treatment)]))

ever_self_employed            log_tot            treated 
                 0                712                  0

In [5]:
# Remove observations missing their censoring time.
dim(data)
summary(data)
skip_vars <- c(names$treatment, names$outcome)
impute <- ck37r::impute_missing_values(data,
                                       skip_vars = skip_vars)

[1] 3891   40

 ever_self_employed    log_tot          treated         treated.1     
 Min.   :0.0000     Min.   : 1.028   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000     1st Qu.: 6.580   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :1.0000     Median : 7.681   Median :1.0000   Median :1.0000  
 Mean   :0.5474     Mean   : 7.593   Mean   :0.7376   Mean   :0.7376  
 3rd Qu.:1.0000     3rd Qu.: 8.645   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.0000     Max.   :11.018   Max.   :1.0000   Max.   :1.0000  
                    NA's   :712                                       
     gender            age        q06_dayorboarding q25_family_business
 Min.   :0.0000   Min.   :20.00   Min.   :0.0000    Min.   :0.0000     
 1st Qu.:0.0000   1st Qu.:22.00   1st Qu.:0.0000    1st Qu.:0.0000     
 Median :0.0000   Median :23.00   Median :1.0000    Median :1.0000     
 Mean   :0.3482   Mean   :23.51   Mean   :0.7396    Mean   :0.5193     
 3rd Qu.:1.0000   3rd Qu.:24.00   3rd Qu.:1.0000    3rd Qu.:1.0000     


In [6]:
# Review missing data for all covariates.
# Only the outcome variable should have missing data at this point.
data <- impute$data
colSums(is.na(data))

ever_self_employed                  log_tot                  treated 
                       0                      712                        0 
               treated.1                   gender                      age 
                       0                        0                        0 
       q06_dayorboarding      q25_family_business       q25a_wk_family_bus 
                       0                        0                        0 
      timeprefs_patience             riskbehavior             mathbusiness 
                       0                        0                        0 
              leadership         perceivedcontrol          timeprefs_delta 
                       0                        0                        0 
          timeprefs_beta        prosocialbehavior                  anxiety 
                       0                        0                        0 
          selfconfidence         big5extroversion         big5emostability 
                       0                        0                        0 
            big5openness        big5conscientious            big5agreeable 
                       0                        0                        0 
        schoolacceptance        currfamwealthstep          tenyrwealthstep 
                       0                        0                        0 
          takingriskstep               ravenscore             father_educ2 
                       0                        0                        0 
            father_educ3             father_educ4             father_educ5 
                       0                        0                        0 
          father_income2           father_income3           mother_income2 
                       0                        0                        0 
          mother_income3               type_house         q13_olevelscore2 
                       0                        0                        0 
       q13_olevelscore34             miss_log_tot   miss_q06_dayorboarding 
                       0                        0                        0 
miss_q25_family_business  miss_q25a_wk_family_bus          miss_leadership 
                       0                        0                        0 
   miss_perceivedcontrol             miss_anxiety      miss_selfconfidence 
                       0                        0                        0 
   miss_schoolacceptance   miss_currfamwealthstep     miss_tenyrwealthstep 
                       0                        0                        0 
     miss_takingriskstep        miss_father_educ2      miss_father_income2 
                       0                        0                        0 
     miss_mother_income2          miss_type_house    miss_q13_olevelscore2 
                       0                        0                        0

In [7]:
## Estimation of causal efects

Y1 <- data$ever_self_employed
Y2 <- data$log_tot[!is.na(data$log_tot)]

A1 <- data$treated
A2 <- data$treated[!is.na(data$log_tot)]

all_covars <- data[, colnames(data) %in% names$covars]

W <- all_covars
W1 <- all_covars
W2 <- subset(data, !is.na(data$log_tot))
W2 <- W2[, colnames(data) %in% names$covars]

screen1 <- screening(x=W1, y=Y1, method="holp", family = "binomial", num.select = 15)$screen
screen2 <- screening(x=W2, y=Y2, method="holp", family = "gaussian", num.select = 15)$screen
screenA <- screening(x=W, y=A1, method="holp", family = "binomial", num.select = 15)$screen 
screenA2 <- screening(x=W2, y=A2, method="holp", family = "binomial", num.select = 15)$screen 

W1 <- W1[,screen1]
W2 <- W2[,screen2]

# William: added/moved this line here to make code work
screenA2 <- screening(x=W2, y=A2, method="holp", family = "binomial", num.select = 15)$screen 

WA <- W[,screenA]
WA2 <- W2[,screenA2]

In [8]:
# Fit glm model (base model, should have the worst performance)

logit2prob <- function(logit){
  odds <- exp(logit)
  prob <- odds / (1 + odds)
  return(prob)
}

model1 <- glm(formula = Y1 ~ A1, family = "binomial")
summary(model1)
logit_control <- model1$coefficients[1]
logit_treated <- model1$coefficients[1] + 1*model1$coefficients[2]
b1 <- logit2prob(logit_treated) - logit2prob(logit_control)
b1

model2 <- glm(formula = Y2 ~ A2, family = "gaussian")
summary(model2)


Call:
glm(formula = Y1 ~ A1, family = "binomial")

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.306  -1.306   1.054   1.054   1.224  

Coefficients:
            Estimate Std. Error z value   Pr(>|z|)    
(Intercept) -0.10784    0.06268  -1.720     0.0854 .  
A1           0.40549    0.07317   5.542 0.00000003 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5359.0  on 3890  degrees of freedom
Residual deviance: 5328.2  on 3889  degrees of freedom
AIC: 5332.2

Number of Fisher Scoring iterations: 4


(Intercept) 
   0.100802


Call:
glm(formula = Y2 ~ A2, family = "gaussian")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-6.6009  -1.0053   0.0917   1.0569   3.4584  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  7.49175    0.05073 147.679   <2e-16 ***
A2           0.13687    0.05895   2.322   0.0203 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 2.123154)

    Null deviance: 6756.7  on 3178  degrees of freedom
Residual deviance: 6745.3  on 3177  degrees of freedom
AIC: 11419

Number of Fisher Scoring iterations: 2


In [9]:
# Define our Super Learner library

g_library <- c("SL.mean",
             "SL.glm",
             "SL.glm.interaction")

Q_library <- c("SL.mean",
             "SL.glm",
             "SL.glm.interaction",
             #"SL.glmnet",
             #"SL.randomForest",
             #"SL.bartMachine",
             "SL.xgboost")

In [10]:
#############################################################
# G-computation formula
#############################################################

np_boot_gcomp <- function(Y, A, W, nrep, family){
  X <- cbind(A,W)
  print(colnames(X))
  # William: wrapped in suppressWarnings() to prevent excessive verbosity
  suppressWarnings(
    QbarSL <- SuperLearner(Y=as.numeric(Y),
                       X=X,
                       SL.library = Q_library,
                       family = family)
    )  
  results <- rep(NA, nrep)
  n <- NROW(Y)
  #stop("stop")
  for(i in 1:nrep){
    i_boot <- sample(1:nrow(W), size = n, replace = TRUE)
    W_boot <- X[i_boot,]
    W1_boot <- W0_boot <- W_boot
    W1_boot$A <- 1
    W0_boot$A <- 0  
    #psi_bootstrap <- G_comp(Y = Y_b, A = A_b, W = W_b, family = family)
    # William: wrapped in suppressWarnings() to prevent excessive verbosity
    suppressWarnings(
      Qbar1W <- predict(QbarSL, newdata=W1_boot, type="response")$pred
      ) 
    # William: wrapped in suppressWarnings() to prevent excessive verbosity
    suppressWarnings(
      Qbar0W <- predict(QbarSL, newdata=W0_boot, type="response")$pred
      ) 
    psi_bootstrap <- (Qbar1W - Qbar0W)
    results[i] <- psi_bootstrap
  }
  return(results)
}

In [11]:
# For business creation
# William: temporary change nrep to 5, so that can test code more quickly
g_comp_boot <- np_boot_gcomp(Y=Y1, A=A1, W=W1, nrep=2, family = "binomial")
summary(g_comp_boot) 
b_iptw <- mean(g_comp_boot)
print(b_iptw)
sd_iptw <- sd(g_comp_boot)
print(sd_iptw)
t_stat <- b_iptw/sd_iptw
p_val <- dt(t_stat, df = n-1, log=FALSE)
print(p_val)
quantile(g_comp_boot, probs = c(0.025,0.975))

 [1] "A"                 "gender"            "age"              
 [4] "q06_dayorboarding" "treated"           "type_house"       
 [7] "mother_income2"    "father_income3"    "father_income2"   
[10] "prosocialbehavior" "big5emostability"  "currfamwealthstep"
[13] "ravenscore"        "big5agreeable"     "leadership"       
[16] "big5openness"     


Warning message in results[i] <- psi_bootstrap:
“number of items to replace is not a multiple of replacement length”Warning message in results[i] <- psi_bootstrap:
“number of items to replace is not a multiple of replacement length”

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.09184 0.09650 0.10116 0.10116 0.10582 0.11047 

[1] 0.1011595
[1] 0.01317395
[1] 0.00000000002843003


2.5%      97.5% 
0.09230988 0.11000913

In [12]:
# For log of total earnings
# William: temporary change to nrep=5 so can can test code more quickly
tot_g_comp_boot <- np_boot_gcomp(Y=Y2, A=A2, W=W2, nrep=2, family = "gaussian") 
summary(tot_g_comp_boot)
b_iptw <- mean(tot_g_comp_boot)
print(b_iptw)
sd_iptw <- sd(tot_g_comp_boot)
print(sd_iptw)
t_stat <- b_iptw/sd_iptw
p_val <- dt(t_stat, df = n-1, log=FALSE)
print(p_val)
quantile(tot_g_comp_boot, probs = c(0.025,0.975))

 [1] "A"                   "gender"              "q13_olevelscore34"  
 [4] "big5emostability"    "timeprefs_delta"     "treated"            
 [7] "q25_family_business" "q06_dayorboarding"   "q13_olevelscore2"   
[10] "age"                 "tenyrwealthstep"     "timeprefs_beta"     
[13] "leadership"          "father_educ4"        "mother_income3"     
[16] "anxiety"            


Warning message in results[i] <- psi_bootstrap:
“number of items to replace is not a multiple of replacement length”Warning message in results[i] <- psi_bootstrap:
“number of items to replace is not a multiple of replacement length”

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1760  0.1875  0.1990  0.1990  0.2105  0.2220 

[1] 0.1989942
[1] 0.03250458
[1] 0.00000004248412


2.5%     97.5% 
0.1771593 0.2208292

In [13]:
#############################################################
# IPTW
#############################################################

iptw <- function(Y, A, X, family){
  n <- NROW(Y)
  print(n)
  # William: wrapped in suppressWarnings() to prevent excessive verbosity
  suppressWarnings(
    propensity_score <- SuperLearner(Y=A,
                   X=X,
                   SL.library = g_library,
                   family = family)
    ) 
  # Obtain predicted probability of treatment 
  # William: wrapped in suppressWarnings() to prevent excessive verbosity
  suppressWarnings(
    pred.g1W <- predict(propensity_score, newX = X, type = 'response')$pred
    ) 
  # Probability of not being treated
  pred.g0W <- 1 - pred.g1W
  # Create vector gAW
  gAW <- rep(NA, n)
  gAW[A==1] <- pred.g1W[A==1]
  gAW[A==0] <- pred.g0W[A==0]
  # Create vector with inverse of predicted probability
  wt <- 1/gAW
  # Implement stabilized IPTW estimator (a.k.a. the modified Horvitz-Thompson estimator)
  Psi_hat <- mean(as.numeric(A==1)*wt*Y)/mean(as.numeric(A==1)*wt) -
             mean(as.numeric(A==0)*wt*Y)/mean(as.numeric(A==0)*wt)
  return(Psi_hat)
  }

np_boot <- function(Y, A, X, family, nrep){
  results <- rep(NA, nrep)
  n <- NROW(Y)
  df <- cbind(Y,A,X)
  for(i in 1:nrep){
    i_boot <- sample(1:nrow(df), size = n, replace = TRUE)
    df_bootstrap <- df[i_boot,]
    Y_b <- df_bootstrap[,1]
    A_b <- df_bootstrap[,2]
    W_b <- subset(df_bootstrap, select = -c(1,2))
    psi_bootstrap <- iptw(Y = Y_b, A = A_b, X = W_b, family = family)
    results[i] <- psi_bootstrap
  }
  return(results)
}

In [14]:
# IPTW for business creation
ate_iptw <- iptw(Y=Y1, A=A1, X=WA, family = "binomial")
ate_iptw

# William: added argument, family = "binomial" -- to avoid error 'argument "family" is missing, with no default'

# William: temporary change to nrep =5 to make code more quickly testable
iptw_bootstrap <- np_boot(Y=Y1, A=A1, X=WA, nrep = 2, family="binomial") 
summary(iptw_bootstrap)
b_iptw <- mean(iptw_bootstrap)
print(b_iptw)
sd_iptw <- sd(iptw_bootstrap)
print(sd_iptw)
t_stat <- b_iptw/sd_iptw
p_val <- dt(t_stat, df = n-1, log=FALSE)
print(p_val)
quantile(iptw_bootstrap, probs = c(0.025,0.975))

[1] 3891


[1] 0.100802

[1] 3891
[1] 3891


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.09234 0.10781 0.12328 0.12328 0.13874 0.15421 

[1] 0.1232772
[1] 0.04374485
[1] 0.008398964


2.5%      97.5% 
0.09389152 0.15266285

In [15]:
# IPTW log total earnings

total_earn_iptw <- iptw(Y=Y2, A=A2, X=WA2, family = "gaussian")
total_earn_iptw

# William: temporary change to nrep=1 for quicker testing
total_iptw_bootstrap <- np_boot(Y=Y2, A=A2, X=WA2, nrep = 2, family = "gaussian") 
summary(total_iptw_bootstrap)
b_iptw <- mean(total_iptw_bootstrap)
print(b_iptw)
sd_iptw <- sd(total_iptw_bootstrap)
print(sd_iptw)
t_stat <- b_iptw/sd_iptw
p_val <- dt(t_stat, df = n-1, log=FALSE)
print(p_val)
quantile(total_iptw_bootstrap, probs = c(0.025,0.975))

[1] 3179


[1] 0.1368721

[1] 3179
[1] 3179


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1011  0.1111  0.1211  0.1211  0.1311  0.1411 

[1] 0.1211078
[1] 0.02828603
[1] 0.00008143755


2.5%     97.5% 
0.1021066 0.1401090

In [16]:
###################################################
# TMLE 
###################################################

# Business creation
tmle <- tmle(Y = as.numeric(Y1),
                A = as.numeric(A1),
                W = W1,
                gform = "A~1",
                family = "binomial",
                #g.SL.library = g_library,
                Q.SL.library = Q_library,
                fluctuation = "logistic") #,
                #V=10)
tmle

 Additive Effect
   Parameter Estimate:  0.10704
   Estimated Variance:  0.00028814
              p-value:  0.00000000028623
    95% Conf Interval: (0.073773, 0.14031) 

 Additive Effect among the Treated
   Parameter Estimate:  0.10704
   Estimated Variance:  0.00028748
              p-value:  0.00000000027325
    95% Conf Interval: (0.07381, 0.14028) 

 Additive Effect among the Controls
   Parameter Estimate:  0.10704
   Estimated Variance:  0.00029013
              p-value:  0.00000000032921
    95% Conf Interval: (0.073658, 0.14043) 

 Relative Risk
   Parameter Estimate:  1.2285
              p-value:  0.0000000028264
    95% Conf Interval: (1.1479, 1.3148) 

              log(RR):  0.20578
    variance(log(RR)):  0.0011996 

 Odds Ratio
   Parameter Estimate:  1.5383
              p-value:  0.00000000032556
    95% Conf Interval: (1.345, 1.7594) 

              log(OR):  0.43067
    variance(log(OR)):  0.0046938 

 Additive Effect
   Parameter Estimate:  0.14211
   Estimated Variance:  0.0029812
              p-value:  0.0092484
    95% Conf Interval: (0.035094, 0.24913) 

 Additive Effect among the Treated
   Parameter Estimate:  0.14211
   Estimated Variance:  0.0029746
              p-value:  0.009171
    95% Conf Interval: (0.035212, 0.24901) 

 Additive Effect among the Controls
   Parameter Estimate:  0.14211
   Estimated Variance:  0.0030015
              p-value:  0.009489
    95% Conf Interval: (0.03473, 0.24949) 

In [ ]:
# Log of total earnings
tot_tmle <- tmle(Y = as.numeric(Y2),
                A = as.numeric(A2),
                W = W2,
                gform = "A~1",
                family = "gaussian",
                #g.SL.library = g_library,
                Q.SL.library = Q_library,
                fluctuation = "logistic") #,
                #V=10)
tot_tmle